In [1]:
import glob
import os

import geopandas as gpd
import pandas as pd
import rasterio
import rasterio.features
import rasterio.warp
from shapely.geometry import box
from tqdm import tqdm

root = "/workspace/storage/data/eurosat/ds/images/remote_sensing/otherDatasets/sentinel_2/tif"
images = glob.glob(os.path.join(root, "**", "*.tif"))
len(images)

27000

In [8]:
data = {}
geoms = []
for image in tqdm(images):
    f = rasterio.open(image)
    data[os.path.basename(image)] = {
        "path": image,
        "filename": os.path.basename(image),
        "label": os.path.basename(os.path.dirname(image)),
    }
    geoms.append(box(*rasterio.warp.transform_bounds(f.crs, "EPSG:4326", *f.bounds)))
    f.close()

df = gpd.GeoDataFrame(pd.DataFrame(data).T, geometry=geoms, crs="EPSG:4326")
df.to_parquet("../data/eurosat.parquet")

100%|██████████| 27000/27000 [00:56<00:00, 475.53it/s]


In [6]:
df = gpd.read_parquet("../data/eurosat.parquet")
df.head()

,index,path,filename,label,geometry
0,Forest_1718.tif,/workspace/storage/data/eurosat/ds/images/remo...,Forest_1718.tif,Forest,"POLYGON ((8.05468 49.35174, 8.05468 49.35757, ..."
1,Forest_2596.tif,/workspace/storage/data/eurosat/ds/images/remo...,Forest_2596.tif,Forest,"POLYGON ((3.85610 50.38271, 3.85610 50.38852, ..."
2,Forest_1586.tif,/workspace/storage/data/eurosat/ds/images/remo...,Forest_1586.tif,Forest,"POLYGON ((12.18449 42.17553, 12.18449 42.18150..."
3,Forest_2845.tif,/workspace/storage/data/eurosat/ds/images/remo...,Forest_2845.tif,Forest,"POLYGON ((27.44272 42.91130, 27.44272 42.91709..."
4,Forest_666.tif,/workspace/storage/data/eurosat/ds/images/remo...,Forest_666.tif,Forest,"POLYGON ((6.14389 49.83781, 6.14389 49.84380, ..."
